In [1]:
import transformers
import tokenizers
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np

In [2]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files='./nsmc/ratings_train.txt', vocab_size=400, min_frequency=2,
                show_progress=True,
                special_tokens=["<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>"])
#Save the Tokenizer to disk
tokenizer.save_model('./distilbert')

['./distilbert\\vocab.json', './distilbert\\merges.txt']

In [3]:
## 이렇게 하면 에러가 뜬다. BertWordPieceTokenizer는 sep_token이 필요하기 때문 
BertWordPieceTokenizer('./distilbert/vocab.json', './distilbert/merges.txt')

NameError: name 'BertWordPieceTokenizer' is not defined

In [4]:
loading_tokenizer = ByteLevelBPETokenizer('./distilbert/vocab.json', './distilbert/merges.txt')

In [5]:
from transformers import DistilBertTokenizer, RobertaTokenizer

distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [6]:
distilbert_tokenizer.cls_token_id ## 이런건 cls_token이 모두 정해져있다. 
roberta_tokenizer.cls_token

101

In [9]:
changing_tokenizer = RobertaTokenizer.from_pretrained('./distilbert')

file ./distilbert\config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [11]:
changing_tokenizer ## 잘 보면, vocab.json/merges.txt만 RobertaTokenizer에 넘겨주었더니 알아서 special token을 added 해주었고 여러가지 설정을 자기가 봐주었다.
### 그렇다고 pretrained된 tokenizer에 추가하는것도 아니다. 내가 가진 파일이 전부인것
### 추가된건 added라고 써있고 added_token으로 확인도 가능하다. 

PreTrainedTokenizer(name_or_path='./distilbert', vocab_size=400, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [16]:
### 이렇게하면 에러뜬다. Distlberttokenize의 경우는 vocab.txt가 필요하기 때문이다. 

changing_tokenizer = DistilBertTokenizer.from_pretrained('./distilbert')

OSError: Can't load tokenizer for './distilbert'. Make sure that:

- './distilbert' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure './distilbert' is not a path to a local directory with something else, in that case)

- or './distilbert' is the correct path to a directory containing relevant tokenizer files



In [17]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path

# Initialize a tokenizer
tokenizer2 = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer2.train(files='./nsmc/ratings_train.txt', vocab_size=400, min_frequency=2,
                show_progress=True,
                special_tokens=['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'])
#Save the Tokenizer to disk
tokenizer2.save_model('./distilbert2')

['./distilbert2\\vocab.json', './distilbert2\\merges.txt']

In [19]:
tokenizer2

Tokenizer(vocabulary_size=400, model=ByteLevelBPE, add_prefix_space=False, lowercase=True, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [20]:
## robertatokenizer는 이렇게 알아서 special token을 추가해준다. 
### 밑에 berttokenizer와의 차이는 roberta는 vocab.json과 merges.txt가 필요한것이고
### bert의 경우에는 vocab.txt가 필요한 것이다. 
token = RobertaTokenizer.from_pretrained('./distilbert2')

file ./distilbert2\config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
token = BertTokenizer.from_pretrained('./distilbert/vocab.json') ### 잘 된것같지만 잘 보면 vocab.json을 햇기 때문에 json 을 하나의 문자열로 읽어서 vocab size=1이다
### 잘못된것. 
print(token.vocab_size)

C:\Users\chkim\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1645: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1


In [25]:
tt = BertTokenizer.from_pretrained('bert-base-uncased')
import os
os.mkdir('./test2')

tt.save_vocabulary('./test2')
tt.save_pretrained('./test') ### save_pretrained를 하면, tokenizer의 config까지 다 같이 저장이 된다. 
### 아래를 보면 알 수 있지만 bert의 경우에는 vocab.txt가 필요한 것이다. 

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: './test2'

In [53]:
### 얘는 ROberta에 맞게 했으므로, special token이 추가가 안되었지만, 
### 밑에 코드는 추가가 되었다. 
vv = RobertaTokenizer.from_pretrained('./distilbert')

file ./distilbert\config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [54]:
kk = RobertaTokenizer.from_pretrained('./distilbert2')

file ./distilbert2\config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


정리하면 BPE, WORDPIECE 뭘 써도 되지만 결과가 VOCAB.TXT, vocab.json인지 잘 확인하고 이걸 다른 roberta.from_pretrained 하면 roberta 맞게 special token도 추가해주고(이미 했으면
안해주고).. 잘 맞춰서 바꿔주는것이다. 